# Google Colab (Jupyter) notebook for Kaggle Airbus Ship detection competition, based on Matterport Mask R-CNN

Results:
*   0.653 starting model with deepretina augmentation
*   RPN_ANCHOR_SCALES = (8,16,32,64,128) 0.598, so keep default 
* 0.658 with default augmentation, with clean_mask
* 0.655 with default augmentation, without clean_mask, and Adam optimizer
* 0.670 train with default augmentation and 400 STEPS_PER_EPOCH
* 0.680 without clean_mask and default augmentation and 400 STEPS_PER_EPOCH, from now without CLEAN_MASK
* 0.687 with iafoss classification file
* from skimage.morphology import dilation, erosion: 0.683->0.669
* from skimage.morphology import dilation: 0.694->0.685
* val set not only 0.jpg images, but even 1.jpg: 0.683->0.688
* results 300 steps, 10 epochs: adam amsgrad: 0.662, sgd: 0.679, adamax: 0.683, adamax retrained 600 steps, 10 epochs, lr 0.00001 and 0.000001: 0.687
* detect with lower val_mrcnn_mask_loss model value instead of with last trained model doesn't increase result
* sgd, 300 steps, epochs lr 10 0.001 10 0.0001 10 0.00001 10 0.000001: 0.679. Retrained 300 steps. epochs lr 10 0.0001 20 0.00001 10 0.000001: 0.694
* binary_fill_holes doesnt change anything
* dropping predicted object masks smaller than 50 pixels, between 50 and 1000 pixels apply the mask minimal rectangle (neptune): 0.694->0.690
* DETECTION_MIN_CONFIDENCE = 0.9: 0.694, 0.8: 0.683; 0.95: 0.670, 0.88:0.694, 0.85: 0.687
* retrain from 0.694 30 epochs 300 step: lr=10^-5: 0.607
* DETECTION_NMS_THRESHOLD = 0.1->0.2: 0.694->0.695, 0.2->0.3: 0.698->0.698, 0.2->0.0: 0.698->0.700
* retrain from 0.694, 10 epochs x 400 steps x lr 10^-4 10^-5 10^-6: 0.694->0.691 
* retrain from 0.694, 20 epochs x 300 steps x lr 10^-5 10^-6: very low, maybe overfitting
* retrain without iaa.GaussianBlur(sigma=(0.0, 5.0)), set SomeOf(0, 5), 10 epochs x 300 steps x lr 10^-4 10^-5 10^-6: 0.694->0.697
* retrain without iaa.GaussianBlur(sigma=(0.0, 5.0)), 10 epochs x 300 steps x lr 10^-4 10^-5 10^-6: 0.694->0.698
* remove prediction ships <= 50 pixels, doesn't change result
* POST_NMS_ROIS_INFERENCE 1000-> 2000, doesn't change result
* retrain from 0.700, 300 steps, 10 epochs 10^-4, 20 10^-5, 10 10^-6: 0.689
* cluster_prediction of mirzaevinom, combined predictions on actual image and horizontally flipped: MIN_CONFIDENCE = 0.9: 0.700->0.696, 0.85: 0.687->0.689
* RPN_ANCHOR_SCALES 8,16,32,64,128: 0.700->0.672 - 16, 32, 64, 128, 256: 0.700->0.634
* trained only with lr 10^-4 and 10^-5 (not 10^-6): 0.700->0.668
* trained from 0.700: 300, 30, 10^-4 10^-5 10^-6: 0.696
* trained from 0.695: 400, 30, 10^-4 10^-5 10^-6: 0.693
* trained from 0.700: 300, 10, 10^-7: very low, maybe overfitting
* tta lr ud: 0.95: only lr 0.699, even ud:0.699, 0.94:0.700, 0.93:0.700
* image augmentation from kaggle hmendonca airbus, 10^-5 10^6 very low, maybe overfitting
* image augmentation from kaggle hmendonca airbus, 10^-4 10^-5 10^6: 0-700->0.697
* add to training set 3-5k images without ships:0.700->0.687
* delete prediction big shape not sharp(rotated_bounding_box): l1+l2, l1/l2: 150, 2.3: 0.701 - 120, 2.3: 0.700 - 150, 2.0: 0.701
* 8-tta merge_cluster_prediction: 0.701->0.704






Todo:
* pretty much every high scoring team used the lovasz hinge loss to increase their score after first training with BCE or focal
* adaboost
* Split (using stratification) train and val dataset



In [0]:
ls /root

#Requirements

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
from google.colab import files

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = os.path.expanduser("~/.kaggle/kaggle.json")
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [0]:
%cd
!git clone --quiet https://github.com/matterport/Mask_RCNN.git

In [0]:
%cd Mask_RCNN

!pip install kaggle
!pip install imgaug
!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

%cd ..

In [0]:
!kaggle competitions download -c airbus-ship-detection

!unzip -q -d train train_v2.zip
!unzip -q train_ship_segmentations_v2.csv.zip

!mkdir test
!mkdir test/ship
!mkdir test/noship
!unzip -q -d test/noship test_v2.zip

!rm *.zip

In [0]:
%cd

import pandas as pd
import shutil
import glob

os.remove('train/6384c3e78.jpg')

delete_files =  []
for f in delete_files:
  os.remove('/root/test/noship/'+f)

traindatadir = '/root/train/'
df = pd.read_csv('train_ship_segmentations_v2.csv')
print('Move',df['ImageId'].nunique(),'file')

ship_dir = traindatadir + 'ship'
os.mkdir(ship_dir)
dfNotNull = df[df.EncodedPixels.notnull()]
imageIdsShip = dfNotNull['ImageId'].unique()
for file_name in imageIdsShip:
  shutil.move(traindatadir + file_name, ship_dir)
print('Moved in',ship_dir,imageIdsShip.size,'file')

#for file_name in glob.glob(traindatadir+"/*5.jpg"):
#  shutil.move(file_name, ship_dir)

noship_dir = traindatadir + 'noship'
os.mkdir(noship_dir)
for file_name in glob.glob(traindatadir+"/*.jpg"):
  shutil.move(file_name, noship_dir)
print('Moved in',noship_dir)

In [0]:
#model_file = 'Mask_RCNN/mrcnn/model.py'

#with open(model_file, 'r') as file:
#  filedata = file.read()

#filedata = filedata.replace('SGD', 'Adamax')
#filedata = filedata.replace('momentum=momentum,', '')
#filedata = filedata.replace('SGD', 'Adam')
#filedata = filedata.replace('momentum=momentum', 'amsgrad=True')

#with open(model_file, 'w') as file:
#  file.write(filedata)


In [0]:
ROOT_DIR = os.path.abspath("/root/")
MASKRCNN_DIR = os.path.join(ROOT_DIR, "Mask_RCNN/")
DATASET_DIR = os.path.join(ROOT_DIR, ".")
TRAIN_DIR = os.path.join(ROOT_DIR, "train")
TRAIN_SHIP_DATASET_DIR = os.path.join(ROOT_DIR, "train/ship/")
TEST_DIR = os.path.join(ROOT_DIR, "test/")
SHIP_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_ship.h5")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
RESULTS_DIR = os.path.join(ROOT_DIR, "results")

import sys
sys.path.append(ROOT_DIR)  # To find local version of the library
sys.path.append(MASKRCNN_DIR)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.data import imread
import matplotlib.pyplot as plt
import datetime
import glob
from PIL import Image
import os
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import visualize
from mrcnn import model as modellib

val_image_ids = []
for file_name in glob.glob(os.path.join(TRAIN_SHIP_DATASET_DIR,"*0.jpg")) + glob.glob(os.path.join(TRAIN_SHIP_DATASET_DIR,"*1.jpg")):
  val_image_ids.append(os.path.splitext(os.path.basename(file_name))[0])

In [0]:
class ShipConfig(Config):
    NAME = "ship"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # Background + ship
    IMAGE_MIN_DIM = 768
    IMAGE_MAX_DIM = 768
    TRAIN_ROIS_PER_IMAGE = 512
    STEPS_PER_EPOCH = 300
    RPN_TRAIN_ANCHORS_PER_IMAGE = 320
#    RPN_ANCHOR_SCALES = (8,16,32,64,128)
    
class InferenceConfig(Config):
    NAME = "ship"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # Background + ship
    IMAGE_MIN_DIM = 768
    IMAGE_MAX_DIM = 768
    TRAIN_ROIS_PER_IMAGE = 512
    MAX_GT_INSTANCES = 20
    DETECTION_MIN_CONFIDENCE = 0.9
    DETECTION_NMS_THRESHOLD = 0.0
    DETECTION_MAX_INSTANCES = 10


############################################################
#  Dataset
############################################################

class ShipDataset(utils.Dataset):

    def load_ship(self, dataset_dir, subset):
        """Load a subset of the ship dataset.

        dataset_dir: Root directory of the dataset
        subset: Subset to load. Either the name of the sub-directory,
                such as stage1_train, stage1_test, ...etc. or, one of:
                * train: stage1_train excluding validation images
                * val: validation images from val_image_ids
        """
        # Add classes. We have one class.
        # Naming the dataset ship, and the class ship
        self.add_class("ship", 1, "ship")

        # Which subset?
        # "val": use hard-coded list above
        # "train": use data from stage1_train minus the hard-coded list above
        # else: use the data from the specified sub-directory
        assert subset in ["train", "val", "test"]
        subset_dir = "train" if subset in ["train", "val"] else subset
        dataset_dir = os.path.join(dataset_dir, subset_dir)
        image_ids = []
        if subset == "val":
            image_ids = val_image_ids
        elif subset == "train":
            # Get image ids from directory names
            masks_count = masks.groupby(['ImageId'], as_index=False).count()
            masks_toget = masks_count.loc[masks_count['EncodedPixels'] >= 1]['ImageId']
            for filename in masks_toget:
                image_ids.append(os.path.splitext(filename)[0])
            image_ids = list(set(image_ids) - set(val_image_ids))
        else:
            for filename in next(os.walk(dataset_dir))[2]:
                image_ids.append(os.path.splitext(filename)[0])
        image_ids.sort()
        # Add images
        for image_id in image_ids:
            self.add_image(
                "ship",
                image_id=image_id,
                path=os.path.join(dataset_dir, "{}.jpg".format(image_id)))
        print('Dataset size:', len(image_ids))

        
    def load_mask(self, image_id):
        image_file = "{}.jpg".format(dataset.image_info[image_id]['id'])
        img_masks = masks.loc[masks['ImageId'] == image_file, 'EncodedPixels'].tolist()
        shape_mask = (768,768)
        
        mask = []
        for img in [x for x in img_masks if not pd.isnull(x)]:
            m = rle_decode(img, shape_mask)
            mask.append(m)
        if (len(mask) > 0):
            mask = np.stack(mask, axis=-1)
        else:
            mask = np.array([])
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)
        
        
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "ship":
            return info["id"]
        else:
            super(self.__class__, self).image_reference(image_id)



############################################################
#  RLE Encoding
############################################################

def rle_encode(mask):
    """Encodes a mask in Run Length Encoding (RLE).
    Returns a string of space-separated values.
    """
    assert mask.ndim == 2, "Mask must be of shape [Height, Width]"
    # Flatten it column wise
    m = mask.T.flatten()
    # Compute gradient. Equals 1 or -1 at transition points
    g = np.diff(np.concatenate([[0], m, [0]]), n=1)
    # 1-based indicies of transition points (where gradient != 0)
    rle = np.where(g != 0)[0].reshape([-1, 2]) + 1
    # Convert second index in each pair to lenth
    rle[:, 1] = rle[:, 1] - rle[:, 0]
    return " ".join(map(str, rle.flatten()))


def rle_decode(rle, shape):
    """Decodes an RLE encoded list of space separated
    numbers and returns a binary mask."""
    rle = list(map(int, rle.split()))
    rle = np.array(rle, dtype=np.int32).reshape([-1, 2])
    rle[:, 1] += rle[:, 0]
    rle -= 1
    mask = np.zeros([shape[0] * shape[1]], np.bool)
    for s, e in rle:
        assert 0 <= s < mask.shape[0]
        assert 1 <= e <= mask.shape[0], "shape: {}  s {}  e {}".format(shape, s, e)
        mask[s:e] = 1
    # Reshape and transpose
    mask = mask.reshape([shape[1], shape[0]]).T
    return mask


def mask_to_rle(image_id, mask, scores):
    "Encodes instance masks to submission format."
    assert mask.ndim == 3, "Mask must be [H, W, count]"
    # If mask is empty, return line with image ID only
    if mask.shape[-1] == 0:
        return "{},".format(image_id)
    # Remove mask overlaps
    # Multiply each instance mask by its score order
    # then take the maximum across the last dimension
    order = np.argsort(scores)[::-1] + 1  # 1-based descending
    mask = np.max(mask * np.reshape(order, [1, 1, -1]), -1)
    # Loop over instance masks
    lines = []
    for o in order:
        m = np.where(mask == o, 1, 0)
        # Skip if empty
        if m.sum() == 0.0:
            continue
        rle = rle_encode(m)
        lines.append("{}, {}".format(image_id, rle))
    return "\n".join(lines)




#Train

In [0]:
%cd

import shutil

for filename in glob.glob(os.path.join(TRAIN_SHIP_DATASET_DIR, '*.*')):
  shutil.copy(filename, TRAIN_DIR)

In [0]:
%cd

train = os.listdir(TRAIN_SHIP_DATASET_DIR)
print(len(train), 'train+val image files with ship')
print('val image files', len(val_image_ids))

masks = pd.read_csv('./train_ship_segmentations_v2.csv')
print(masks['ImageId'].value_counts().shape[0], 'images found in csv, an image can have multiple masks for multiple ship, or not ship (NaN EncodedPixels)')
print(masks['EncodedPixels'].value_counts().shape[0], 'ships in train images')
print(masks.shape[0], 'masks found in csv, included NaN EncodedPixels for images without ship')

masks_count = masks.groupby(['ImageId'], as_index=False).count()
print(len(masks_count.loc[masks_count['EncodedPixels'] == 1]['ImageId']), 'images with zero or one mask')

masks.head()

In [0]:
EPOCHS = 10

############################################################
#  Training
############################################################
from imgaug import augmenters as iaa

def train(model, dataset_dir, subset):
    """Train the model."""
    # Training dataset.
    dataset_train = ShipDataset()
    dataset_train.load_ship(dataset_dir, subset)
    dataset_train.prepare()

    # Validation dataset
    dataset_val = ShipDataset()
    dataset_val.load_ship(dataset_dir, "val")
    dataset_val.prepare()

# Default Image augmentation
    augmentation = iaa.SomeOf((0, 2), [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.OneOf([iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
        iaa.Multiply((0.8, 1.5))
    ])

# Deepretina
#    augmentation = iaa.Sequential([
#        iaa.Fliplr(0.5),
#        iaa.Flipud(0.5),
#        iaa.OneOf([iaa.Affine(rotate=0),
#                   iaa.Affine(rotate=90),
#                   iaa.Affine(rotate=180),
#                   iaa.Affine(rotate=270)]),
#        iaa.Sometimes(0.5,iaa.Affine(rotate=(-10,10))),
#        iaa.Add((-15, 15), per_channel=1)
#    ])
    
    lr = 10**-3
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*1,
                augmentation=augmentation,
                layers='all')
        
    lr = 10**-4
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*2,
                augmentation=augmentation,
                layers='all')
    
    lr = 10**-5
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*3,
                augmentation=augmentation,
                layers='all')

    lr = 10**-6
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*4,
                augmentation=augmentation,
                layers='all')
    
    
    lr = 10**-4
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*5,
                augmentation=augmentation,
                layers='all')
    
    lr = 10**-5
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*7,
                augmentation=augmentation,
                layers='all')

    lr = 10**-6
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*8,
                augmentation=augmentation,
                layers='all')


    lr = 10**-4
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*9,
                augmentation=augmentation,
                layers='all')
    
    lr = 10**-5
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*10,
                augmentation=augmentation,
                layers='all')

    lr = 10**-6
    print("TRAIN WITH LEARNING_RATE", lr)
    model.train(dataset_train, dataset_val,
                learning_rate=lr,
                epochs=EPOCHS*11,
                augmentation=augmentation,
                layers='all')



In [0]:
# Load dataset
dataset = ShipDataset()
# The subset is the name of the sub-directory, such as stage1_train,
# stage1_test, ...etc. You can also use these special values:
#     train: loads stage1_train but excludes validation images
#     val: loads validation images from stage1_train. For a list
#          of validation images see ship.py
dataset.load_ship(DATASET_DIR, subset="train")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [0]:
# Load and display random samples
image_ids = np.random.choice(dataset.image_ids, 4)
for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names, limit=1)

In [0]:
#TO START FROM MY PRETRAINED MODEL, UNCOMMENT 2 LAST LINES

%cd

modelFileId = '1hvp1gy4zEKwpMzfd6pBagNbpgVz4reLu'
modelFilePath = 'mask_rcnn_ship.h5'

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': modelFileId})
downloaded.GetContentFile(modelFilePath)
print('Files downloaded')

In [0]:
weights_path = SHIP_WEIGHTS_PATH
print(weights_path)

config = ShipConfig()
if not os.path.exists(weights_path):
    print("Downloading weights")
    utils.download_trained_weights(weights_path)
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

train(model=model, dataset_dir=DATASET_DIR, subset="train")

In [0]:
last_weights_path = sorted(glob.glob("/root/logs/*/mask_rcnn_*.h5"))[-1]
print(last_weights_path)

from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
    }
    media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)
    created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()
    return created

created = save_file_to_drive('mask_rcnn_ship.h5', last_weights_path)
modelDriveId = created.get('id')
print('File ID: {}'.format(modelDriveId))

#Detect

In [0]:
%cd

#modelFileId = '1hvp1gy4zEKwpMzfd6pBagNbpgVz4reLu' #0.700
modelFileId = modelDriveId
modelFilePath = 'mask_rcnn_ship.h5'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': modelFileId})
downloaded.GetContentFile(modelFilePath)
print('Files downloaded')

In [0]:
%cd

inferenceClassificationId = '1uMpxrf_7_Vx4QIt9uxJ5k2_skluf-lMV'
inferenceClassificationPath = 'inference_classification.csv'

downloaded = drive.CreateFile({'id': inferenceClassificationId})
downloaded.GetContentFile(inferenceClassificationPath)
print('Files downloaded')

import shutil

df = pd.read_csv(inferenceClassificationPath)
files_ship = df[df.prediction == 'ship']
for filename in files_ship.filename:
  shutil.copy(os.path.join(TEST_DIR, 'noship', filename), TEST_DIR)

In [0]:
#%cd

#cropClassificationId = '1Ny12X2plITsrrEd1WdAwo_q2X8I1AZ6a'
#cropClassificationPath = 'crop_classification.csv'

#downloaded = drive.CreateFile({'id': cropClassificationId})
#downloaded.GetContentFile(cropClassificationPath)
#print('Files downloaded')


#class_imgs_crop = pd.read_csv(cropClassificationPath)

#def clean_mask(masks, image_file): #masks = [H, W, N] instance binary masks
#  windows_to_crop = class_imgs_crop[(class_imgs_crop['predict']==0) & (class_imgs_crop['orig_filename']==image_file) & (class_imgs_crop['filename'].str.len()==15)]
#  clean_matrix = np.ones((masks.shape))
#  for row in windows_to_crop.itertuples():
#    crop_num = int(row.filename[-1])
#    if crop_num == 1:
#      clean_matrix[0:368,0:368,:] = 0
#    if crop_num == 2:
#      clean_matrix[0:368,400:,:]  = 0
#    if crop_num == 3:
#      clean_matrix[400:,0:368,:]  = 0
#    if crop_num == 4:
#      clean_matrix[400:,400:,:]   = 0
#  masks = np.multiply(masks, clean_matrix)
#  return masks


#DROP_SIZE = 50
#MID_MIN_SIZE = 50
#MID_MAX_SIZE = 1000

#import cv2

#def mask_to_bbox(mask):
#    img_box = np.zeros_like(mask)
#    _, cnt, _ = cv2.findContours(mask, 1, 2)
#    rect = cv2.minAreaRect(cnt[0])
#    box = cv2.boxPoints(rect)
#    box = np.int0(box)
#    cv2.drawContours(img_box, [box], 0, 1, -1)
#    return img_box

#def mask_postprocessing(masks, image_file): #masks = [H, W, N] instance binary masks
#  h, w, n = masks.shape
#  for i in range(n):
#    mask  = masks[:,:,i]
#    size = np.count_nonzero(mask)
#    if (size <= DROP_SIZE):
#      print('DROP', image_file, i, size)
#      masks[:,:,i] = np.zeros((h, w))
#    elif MID_MIN_SIZE < size < MID_MAX_SIZE:
#      print('MID', image_file, i, size)
#      masks[:,:,i] = mask_to_bbox(mask.astype(np.uint8))
#  return masks


In [0]:
def ensemble_prediction(model, image):

    """ Test time augmentation method using non-maximum supression"""

    masks = []
    scores = []
    boxes = []

    results = {}

    result = model.detect([image], verbose=0)[0]
    masks.append(result['masks'])
    scores.append(result['scores'])
    boxes.append(utils.extract_bboxes(result['masks']))

    temp_img = np.fliplr(image)
    result = model.detect([temp_img], verbose=0)[0]
    mask = np.fliplr(result['masks'])
    masks.append(mask)
    scores.append(result['scores'])
    boxes.append(utils.extract_bboxes(mask))

    temp_img = np.flipud(image)
    result = model.detect([temp_img], verbose=0)[0]
    mask = np.flipud(result['masks'])
    masks.append(mask)
    scores.append(result['scores'])
    boxes.append(utils.extract_bboxes(mask))

    angle = np.random.choice([1, -1])
    temp_img = np.rot90(image, k=angle, axes=(0, 1))
    result = model.detect([temp_img], verbose=0)[0]
    mask = np.rot90(result['masks'], k=-angle, axes=(0, 1))
    masks.append(mask)
    scores.append(result['scores'])
    boxes.append(extract_bboxes(mask))

    masks = np.concatenate(masks, axis=-1)
    scores = np.concatenate(scores, axis=-1)
    boxes = np.concatenate(boxes, axis=0)

    if (boxes.shape[0] > 0):
      keep_ind = utils.non_max_suppression(boxes, scores, 0.1)
      masks = masks[:, :, keep_ind]
      scores = scores[keep_ind]

    results['masks'] = masks
    results['scores'] = scores

    return result
  
def cluster_prediction(model, image):

    """ Test time augmentation method using bounding box IoU"""
    # from utils import non_max_suppression, extract_bboxes, compute_overlaps
    height, width = image.shape[:2]

    # Predict masks on actual image
    result1 = model.detect([image], verbose=0)[0]
    # Handles no mask predictions
    if result1['masks'].shape[0] == 0:
        result1['masks'] = np.zeros([height, width, 1])
        result1['masks'][0, 0, 0] = 1
        result1['scores'] = np.ones(1)

    # Predict masks on LR flipped image
    temp_img = np.fliplr(image)
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.fliplr(result2['masks'])

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm].size != 0 and np.max(overlaps[mm]) > 0.1:
            ind = np.argmax(overlaps[mm])
            mask = result1['masks'][:, :, mm] + result2['masks'][:, :, ind]
            result1['masks'][:, :, mm] = (mask > 0).astype(np.uint8)
        else:
            result1['masks'][:, :, mm] = 0

    return result1


In [0]:
def merge_cluster_prediction(model, image):

    """ Test time augmentation method using bounding box IoU"""
    # from utils import non_max_suppression, extract_bboxes, compute_overlaps
    height, width = image.shape[:2]
    class_names = ['BG', 'ship']
    
    instances_thresh = 0.2
    instances_occur = 4
    
    # Predict masks on actual image
    result1 = model.detect([image], verbose=0)[0]

    # Handles no mask predictions
    if result1['masks'].shape[0] == 0:
        result1['masks'] = np.zeros([height, width, 1])
        result1['masks'][0, 0, 0] = 1
        result1['scores'] = np.ones(1)
        
    instances = np.ones((result1['masks'].shape[2]))
        

      
        
    # Predict masks on LR flipped image
    temp_img = np.fliplr(image)
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.fliplr(result2['masks'])

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm,:].size != 0 and np.max(overlaps[mm,:]) > instances_thresh:
          instances[mm]=instances[mm]+1
    
    for mm in range(overlaps.shape[1]):
        if overlaps[:,mm].size != 0 and np.max(overlaps[:,mm]) == 0.0:
#            print('=========== LR')
            m2 = np.expand_dims(result2['masks'][:, :, mm], axis=2)
            result1['masks'] = np.append(result1['masks'], m2, axis = 2)
            s2 = np.expand_dims(result2['scores'][mm], axis=0)
            result1['scores'] = np.append(result1['scores'], s2, axis = 0)
            r2 = utils.extract_bboxes(m2)
            result1['rois'] = np.append(result1['rois'], r2, axis = 0)
            c2 = np.expand_dims(result2['class_ids'][mm], axis=0)
            result1['class_ids'] = np.append(result1['class_ids'], c2, axis = 0)
            instances = np.append(instances, [1])
            
            
            
            
    # Predict masks on UD flipped image
    temp_img = np.flipud(image)
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.flipud(result2['masks'])

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm,:].size != 0 and np.max(overlaps[mm,:]) > instances_thresh:
          instances[mm]=instances[mm]+1
    
    for mm in range(overlaps.shape[1]):
        if overlaps[:,mm].size != 0 and np.max(overlaps[:,mm]) == 0.0:
#            print('=========== UD')
            m2 = np.expand_dims(result2['masks'][:, :, mm], axis=2)
            result1['masks'] = np.append(result1['masks'], m2, axis = 2)
            s2 = np.expand_dims(result2['scores'][mm], axis=0)
            result1['scores'] = np.append(result1['scores'], s2, axis = 0)
            r2 = utils.extract_bboxes(m2)
            result1['rois'] = np.append(result1['rois'], r2, axis = 0)
            c2 = np.expand_dims(result2['class_ids'][mm], axis=0)
            result1['class_ids'] = np.append(result1['class_ids'], c2, axis = 0)
            instances = np.append(instances, [1])
            
            
            
            
               
    # Predict masks on LR-UD flipped image
    temp_img = np.fliplr(np.flipud(image))
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.fliplr(np.flipud(result2['masks']))

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm,:].size != 0 and np.max(overlaps[mm,:]) > instances_thresh:
          instances[mm]=instances[mm]+1

    for mm in range(overlaps.shape[1]):
        if overlaps[:,mm].size != 0 and np.max(overlaps[:,mm]) == 0.0:
#            print('=========== LR-UD')
            m2 = np.expand_dims(result2['masks'][:, :, mm], axis=2)
            result1['masks'] = np.append(result1['masks'], m2, axis = 2)
            s2 = np.expand_dims(result2['scores'][mm], axis=0)
            result1['scores'] = np.append(result1['scores'], s2, axis = 0)
            r2 = utils.extract_bboxes(m2)
            result1['rois'] = np.append(result1['rois'], r2, axis = 0)
            c2 = np.expand_dims(result2['class_ids'][mm], axis=0)
            result1['class_ids'] = np.append(result1['class_ids'], c2, axis = 0)
            instances = np.append(instances, [1])
            
            
            

    # Predict masks on rot90 no flipped image
    temp_img = np.rot90(image)
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.rot90(result2['masks'], k=-1)

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm,:].size != 0 and np.max(overlaps[mm,:]) > instances_thresh:
          instances[mm]=instances[mm]+1

    for mm in range(overlaps.shape[1]):
        if overlaps[:,mm].size != 0 and np.max(overlaps[:,mm]) == 0.0:
#            print('=========== rot90')
            m2 = np.expand_dims(result2['masks'][:, :, mm], axis=2)
            result1['masks'] = np.append(result1['masks'], m2, axis = 2)
            s2 = np.expand_dims(result2['scores'][mm], axis=0)
            result1['scores'] = np.append(result1['scores'], s2, axis = 0)
            r2 = utils.extract_bboxes(m2)
            result1['rois'] = np.append(result1['rois'], r2, axis = 0)
            c2 = np.expand_dims(result2['class_ids'][mm], axis=0)
            result1['class_ids'] = np.append(result1['class_ids'], c2, axis = 0)
            instances = np.append(instances, [1])
                        
            
            
            
    # Predict masks on rot90 LR flipped image
    temp_img = np.rot90(np.fliplr(image))
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.fliplr(np.rot90(result2['masks'], k=-1))

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm,:].size != 0 and np.max(overlaps[mm,:]) > instances_thresh:
          instances[mm]=instances[mm]+1

    for mm in range(overlaps.shape[1]):
        if overlaps[:,mm].size != 0 and np.max(overlaps[:,mm]) == 0.0:
#            print('=========== rot90-LR')
            m2 = np.expand_dims(result2['masks'][:, :, mm], axis=2)
            result1['masks'] = np.append(result1['masks'], m2, axis = 2)
            s2 = np.expand_dims(result2['scores'][mm], axis=0)
            result1['scores'] = np.append(result1['scores'], s2, axis = 0)
            r2 = utils.extract_bboxes(m2)
            result1['rois'] = np.append(result1['rois'], r2, axis = 0)
            c2 = np.expand_dims(result2['class_ids'][mm], axis=0)
            result1['class_ids'] = np.append(result1['class_ids'], c2, axis = 0)
            instances = np.append(instances, [1])
            
            
            
            
    # Predict masks on rot90 UD flipped image
    temp_img = np.rot90(np.flipud(image))
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.flipud(np.rot90(result2['masks'], k=-1))

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm,:].size != 0 and np.max(overlaps[mm,:]) > instances_thresh:
          instances[mm]=instances[mm]+1

    for mm in range(overlaps.shape[1]):
        if overlaps[:,mm].size != 0 and np.max(overlaps[:,mm]) == 0.0:
#            print('=========== rot90-UD')
            m2 = np.expand_dims(result2['masks'][:, :, mm], axis=2)
            result1['masks'] = np.append(result1['masks'], m2, axis = 2)
            s2 = np.expand_dims(result2['scores'][mm], axis=0)
            result1['scores'] = np.append(result1['scores'], s2, axis = 0)
            r2 = utils.extract_bboxes(m2)
            result1['rois'] = np.append(result1['rois'], r2, axis = 0)
            c2 = np.expand_dims(result2['class_ids'][mm], axis=0)
            result1['class_ids'] = np.append(result1['class_ids'], c2, axis = 0)
            instances = np.append(instances, [1])
            
            
            
            
            
    # Predict masks on rot90 LR-UD flipped image
    temp_img = np.rot90(np.fliplr(np.flipud(image)))
    result2 = model.detect([temp_img], verbose=0)[0]
    result2['masks'] = np.flipud(np.fliplr(np.rot90(result2['masks'], k=-1)))

    # Handles no mask predictions
    if result2['masks'].shape[0] == 0:
        result2['masks'] = np.zeros([height, width, 1])
        result2['masks'][0, 0, 0] = 1
        result2['scores'] = np.ones(1)

    # Compute IoU on masks
    overlaps = utils.compute_overlaps_masks(result1['masks'], result2['masks'])

    for mm in range(overlaps.shape[0]):
        if overlaps[mm,:].size != 0 and np.max(overlaps[mm,:]) > instances_thresh:
          instances[mm]=instances[mm]+1

    for mm in range(overlaps.shape[1]):
        if overlaps[:,mm].size != 0 and np.max(overlaps[:,mm]) == 0.0:
#            print('=========== rot90-LR-UD')
            m2 = np.expand_dims(result2['masks'][:, :, mm], axis=2)
            result1['masks'] = np.append(result1['masks'], m2, axis = 2)
            s2 = np.expand_dims(result2['scores'][mm], axis=0)
            result1['scores'] = np.append(result1['scores'], s2, axis = 0)
            r2 = utils.extract_bboxes(m2)
            result1['rois'] = np.append(result1['rois'], r2, axis = 0)
            c2 = np.expand_dims(result2['class_ids'][mm], axis=0)
            result1['class_ids'] = np.append(result1['class_ids'], c2, axis = 0)
            instances = np.append(instances, [1])
            
            


            
#    print(instances)
    for mm in range(instances.shape[0]):
        if(instances[mm]<instances_occur):
          result1['masks'][:, :, mm] = 0
          result1['rois'][mm] = utils.extract_bboxes(np.expand_dims(result1['masks'][:, :, mm], axis=2))
#          print('INSTANCES',mm)
            
            
            
    return result1

In [0]:
import cv2

class_names = ['BG', 'ship']

def rotated_bounding_box(r, idx, image):
  for mm in range(r['masks'].shape[2]):
    img = r['masks'][:, :, mm]
    img = np.array(img * 255, dtype = np.uint8)
    _, contours,_ = cv2.findContours(img, 1, 2)
    if len(contours) > 0:
      cnt = contours[0]
      rect = cv2.minAreaRect(cnt)
      box = cv2.boxPoints(rect)
      box = np.int0(box)
      l1 = cv2.norm(box[0], box[1], normType=cv2.NORM_L2)
      l2 = cv2.norm(box[1], box[2], normType=cv2.NORM_L2)

      if l1+l2>150 and max(l1,l2)/min(l1,l2)<2.5:
        #print(idx)
        #print(r['scores'][mm])
        #visualize.display_instances(
        #    image, r['rois'], r['masks'], r['class_ids'],
        #    class_names, r['scores'],
        #    show_bbox=True, show_mask=False,
        #    title="Prediction ORIGINAL")
        r['masks'][:, :, mm] = 0
        r['rois'][mm] = utils.extract_bboxes(np.expand_dims(r['masks'][:, :, mm], axis=2))
  return r

In [0]:
#VISUALIZE SOME DETECTIONS

detect_filename = os.path.join(RESULTS_DIR, 'detect_masks.csv')

def detect(model, dataset_dir, subset):
    """Run detection on images in the given directory."""
    print("Running on {}".format(dataset_dir))

    # Create directory
    if not os.path.exists(RESULTS_DIR):
        os.makedirs(RESULTS_DIR)

    # Read dataset
    dataset = ShipDataset()
    dataset.load_ship(dataset_dir, subset)
    dataset.prepare()
    # Load over images
    submission = []
    
    #for idx in range(50,70):
    for idx in [58,92,97,180,322,338,473,521,586,1028]:
        image_id = dataset.image_ids[idx]
        if (idx % 100==0):
            print('Processed images: ', idx, ' - Time: ', datetime.datetime.now())
        # Load image and run detection
        image_file = dataset.image_info[image_id]["id"]+'.jpg'
        image = dataset.load_image(image_id)

        r = model.detect([image], verbose=0)[0]
        visualize.display_instances(
        image, r['rois'], r['masks'], r['class_ids'],
        dataset.class_names, r['scores'],
        show_bbox=True, show_mask=False,
        title="Prediction ORIGINAL", captions=['0','1','2','3','4','5','6','7','8','9'])
        
        r = merge_cluster_prediction(model, image)
        #r = rotated_bounding_box(r, idx, image)
        visualize.display_instances(
        image, r['rois'], r['masks'], r['class_ids'],
        dataset.class_names, r['scores'],
        show_bbox=True, show_mask=False,
        title="Prediction NEW")

        
config = InferenceConfig()
#config.display()

#modelFilePath = model.find_last()
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=DEFAULT_LOGS_DIR)

print("Loading weights ", modelFilePath)
model.load_weights(modelFilePath, by_name=True)

#detect(model=model, dataset_dir=DATASET_DIR, subset="val")
detect(model=model, dataset_dir=DATASET_DIR, subset="test")

In [0]:
# RUN DETECTION

detect_filename = os.path.join(RESULTS_DIR, 'detect_masks.csv')

def detect(model, dataset_dir, subset):
    """Run detection on images in the given directory."""
    print("Running on {}".format(dataset_dir))

    # Create directory
    if not os.path.exists(RESULTS_DIR):
        os.makedirs(RESULTS_DIR)

    # Read dataset
    dataset = ShipDataset()
    dataset.load_ship(dataset_dir, subset)
    dataset.prepare()
    # Load over images
    submission = []
    last_idx = len(dataset.image_ids)
    print('Dataset length:',last_idx)
    for idx in range(last_idx):
        image_id = dataset.image_ids[idx]
        if (idx % 20==0):
            print('Processed images: ', idx, ' - Time: ', datetime.datetime.now())
        # Load image and run detection
        image = dataset.load_image(image_id)
        image_file = dataset.image_info[image_id]["id"]+'.jpg'
        # Detect objects
        #r = model.detect([image], verbose=0)[0]
        r = merge_cluster_prediction(model, image)
        
        r = rotated_bounding_box(r, idx, image)

        # Encode image to RLE. Returns a string of multiple lines
        rle = mask_to_rle(image_file, r["masks"], r["scores"])
        submission.append(rle)
        # Save image with masks
#        visualize.display_instances(
#            image, r['rois'], r['masks'], r['class_ids'],
#            dataset.class_names, r['scores'],
#            show_bbox=False, show_mask=False,
#            title="Predictions")
        plt.savefig("{}/{}.png".format(RESULTS_DIR, dataset.image_info[image_id]["id"]))

    # Save to csv file
    submission = "ImageId,EncodedPixels\n" + "\n".join(submission)
    with open(detect_filename, "w") as f:
        f.write(submission)
    print("Saved to", detect_filename)

In [0]:
config = InferenceConfig()
config.display()

#modelFilePath = model.find_last()
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=DEFAULT_LOGS_DIR)

print("Loading weights ", modelFilePath)
model.load_weights(modelFilePath, by_name=True)

#detect(model=model, dataset_dir=DATASET_DIR, subset="val")
detect(model=model, dataset_dir=DATASET_DIR, subset="test")

In [0]:
from shutil import copy

submit_file = 'submit.csv'
copy(detect_filename, submit_file)

df = pd.read_csv(submit_file)
exludedAndProcessedFiles = df['ImageId'].unique().tolist()
print(len(exludedAndProcessedFiles))
noship_files = ''
test_files = next(os.walk('test/noship'))[2]
print(len(test_files))
for filename in test_files:
    if (filename not in exludedAndProcessedFiles):
        noship_files+='\n'+filename+','

with open(submit_file, 'a') as f:
    f.write(noship_files)
print("Saved to ", submit_file)
files.download(submit_file)
files.download(detect_filename)


In [0]:
#VISUALIZE TEST

%cd

from IPython.display import Image, display

masks = pd.read_csv(detect_filename)
print(masks['ImageId'].value_counts().shape[0], 'images found in csv, an image can have multiple masks for multiple ship, or not ship (NaN EncodedPixels)')



# Load dataset
dataset = ShipDataset()
dataset.load_ship(DATASET_DIR, subset="test")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    
image_ids = np.random.choice(dataset.image_ids, 4)
for image_id in image_ids:
    image_file = "{}.jpg".format(dataset.image_info[image_id]['id'])
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    print(image_file, class_ids.size)
    if class_ids.size != 0:
        visualize.display_top_masks(image, mask, class_ids, dataset.class_names, limit=1)
    else:
        visualize.display_images([image], cols = 2)